# NESS: Google Colab Tutorial

In [ ]:
# install NESS Library

!pip install ness-search biopython pyMSA

# install MUSCLE

!apt install muscle ncbi-blast+

In [1]:
from Bio import SeqIO
from Bio.Blast import NCBIXML
from IPython.display import HTML
from pymsa import MSA, Entropy, PercentageOfNonGaps, PercentageOfTotallyConservedColumns, Star, SumOfPairs
from pymsa import PAM250, Blosum62, FileMatrix
from pymsa.util.fasta import print_alignment
from scipy.stats import kruskal
from plotly import express as ex
import pandas as pd
import random
import os

## Downloading 

To ilustrate the usage of NESS, let's download a list of protein sequences from the Protein Data Bank (PDB) database.

In [2]:
!mkdir -p data/
!wget -O data/pdb_seqres.txt.gz https://files.wwpdb.org/pub/pdb/derived_data/pdb_seqres.txt.gz
!gzip -d -f data/pdb_seqres.txt.gz
!mv data/pdb_seqres.txt data/pdb_seqres.fasta

--2022-10-29 01:27:38--  https://files.wwpdb.org/pub/pdb/derived_data/pdb_seqres.txt.gz
Resolving files.wwpdb.org (files.wwpdb.org)... 128.6.158.70
Connecting to files.wwpdb.org (files.wwpdb.org)|128.6.158.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42533197 (41M) [application/x-gzip]
Saving to: ‘data/pdb_seqres.txt.gz’

data/pdb_seqres.txt 100%[===================>]  40,56M  6,97MB/s    in 19s     

2022-10-29 01:27:58 (2,11 MB/s) - ‘data/pdb_seqres.txt.gz’ saved [42533197/42533197]



In [3]:
!head -10 data/pdb_seqres.fasta

>100d_A mol:na length:10  DNA/RNA (5'-R(*CP*)-D(*CP*GP*GP*CP*GP*CP*CP*GP*)-R(*G)-3')
CCGGCGCCGG
>100d_B mol:na length:10  DNA/RNA (5'-R(*CP*)-D(*CP*GP*GP*CP*GP*CP*CP*GP*)-R(*G)-3')
CCGGCGCCGG
>101d_A mol:na length:12  DNA (5'-D(*CP*GP*CP*GP*AP*AP*TP*TP*(CBR)P*GP*CP*G)-3')
CGCGAATTCGCG
>101d_B mol:na length:12  DNA (5'-D(*CP*GP*CP*GP*AP*AP*TP*TP*(CBR)P*GP*CP*G)-3')
CGCGAATTCGCG
>101m_A mol:protein length:154  MYOGLOBIN
MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDRVKHLKTEAEMKASEDLKKHGVTVLTALGAILKKKGHHEAELKPLAQSHATKHKIPIKYLEFISEAIIHVLHSRHPGNFGADAQGAMNKALELFRKDIAAKYKELGYQG


As PDB also includes data from non-protein molecules (eg: DNA, RNA), let's filter only the protein sequences using the field `mol:protein` present in the FASTA header.

In [4]:
with open('data/pdb_seqres.fasta') as reader, open('data/pdb_seqres_proteins_only.fasta','w') as writer:
    for r, record in enumerate(SeqIO.parse(reader, 'fasta')):
        if 'mol:protein' in record.description:
            SeqIO.write([record], writer, 'fasta')

In [5]:
!head -10 data/pdb_seqres_proteins_only.fasta

>101m_A mol:protein length:154  MYOGLOBIN
MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDRVKHLKTEAEMKASE
DLKKHGVTVLTALGAILKKKGHHEAELKPLAQSHATKHKIPIKYLEFISEAIIHVLHSRH
PGNFGADAQGAMNKALELFRKDIAAKYKELGYQG
>102l_A mol:protein length:165  T4 LYSOZYME
MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAAKSELDKAIGRNTNGVIT
KDEAEKLFNQDVDAAVRGILRNAKLKPVYDSLDAVRRAALINMVFQMGETGVAGFTNSLR
MLQQKRWDEAAVNLAKSRWYNQTPNRAKRVITTFRTGTWDAYKNL
>102m_A mol:protein length:154  MYOGLOBIN
MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDRFKHLKTEAEMKASE


## Creating sequence vectorization model

Now, we will create a protein sequence vectorization model based on the [Word2Vec algorithm](https://arxiv.org/abs/1301.3781). The model will be trained for `1 epoch` and considering a k-mer size with `k=5`. 

In [6]:
!ness build_model --help

usage: ness build_model [-h] --input INPUT --output OUTPUT
                        [--model-type {word2vec}] [--vector-size VECTOR_SIZE]
                        [--window-size WINDOW_SIZE] [--min-count MIN_COUNT]
                        [--ksize KSIZE] [--threads THREADS] [--epochs EPOCHS]
                        [--corpus-file CORPUS_FILE] [--both-strands] [--debug]

optional arguments:
  -h, --help            show this help message and exit
  --input INPUT         input file in FASTA format
  --output OUTPUT
  --model-type {word2vec}
  --vector-size VECTOR_SIZE
  --window-size WINDOW_SIZE
  --min-count MIN_COUNT
  --ksize KSIZE
  --threads THREADS
  --epochs EPOCHS
  --corpus-file CORPUS_FILE
  --both-strands
  --debug


In [ ]:
!ness build_model \
  --input data/pdb_seqres_proteins_only.fasta \
  --ksize 5 \
  --epochs 10 \
  --output data/ness_pdb.model \
  --debug

 
       ▄▄    ▄ ▄▄▄▄▄▄▄ ▄▄▄▄▄▄▄ ▄▄▄▄▄▄▄ 
      █  █  █ █       █       █       █
      █   █▄█ █    ▄▄▄█  ▄▄▄▄▄█  ▄▄▄▄▄█
      █       █   █▄▄▄█ █▄▄▄▄▄█ █▄▄▄▄▄ 
      █  ▄    █    ▄▄▄█▄▄▄▄▄  █▄▄▄▄▄  █
      █ █ █   █   █▄▄▄ ▄▄▄▄▄█ █▄▄▄▄▄█ █
      █▄█  █▄▄█▄▄▄▄▄▄▄█▄▄▄▄▄▄▄█▄▄▄▄▄▄▄█      

Vector-based Alignment-free Sequence Search

2022-10-29 01:28:32,604 : INFO : Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=100, alpha=0.025>', 'datetime': '2022-10-29T01:28:32.603788', 'gensim': '4.2.0', 'python': '3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) \n[GCC 9.4.0]', 'platform': 'Linux-5.15.0-52-generic-x86_64-with-debian-bookworm-sid', 'event': 'created'}
2022-10-29 01:33:07,940 : INFO : collecting all words and their counts
2022-10-29 01:33:07,940 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-10-29 01:33:07,998 : INFO : PROGRESS: at sentence #1000, processed 205721 words, keeping 65766 word types
2022-10-29 01:33:0

2022-10-29 01:33:12,789 : INFO : PROGRESS: at sentence #65000, processed 15293528 words, keeping 1242119 word types
2022-10-29 01:33:12,859 : INFO : PROGRESS: at sentence #66000, processed 15513053 words, keeping 1251384 word types
2022-10-29 01:33:12,933 : INFO : PROGRESS: at sentence #67000, processed 15741008 words, keeping 1262075 word types
2022-10-29 01:33:13,013 : INFO : PROGRESS: at sentence #68000, processed 15993244 words, keeping 1270933 word types
2022-10-29 01:33:13,093 : INFO : PROGRESS: at sentence #69000, processed 16227910 words, keeping 1279759 word types
2022-10-29 01:33:13,176 : INFO : PROGRESS: at sentence #70000, processed 16464576 words, keeping 1290450 word types
2022-10-29 01:33:13,259 : INFO : PROGRESS: at sentence #71000, processed 16719866 words, keeping 1301259 word types
2022-10-29 01:33:13,333 : INFO : PROGRESS: at sentence #72000, processed 16963010 words, keeping 1307137 word types
2022-10-29 01:33:13,418 : INFO : PROGRESS: at sentence #73000, processed

2022-10-29 01:33:18,544 : INFO : PROGRESS: at sentence #136000, processed 32336832 words, keeping 1758653 word types
2022-10-29 01:33:18,623 : INFO : PROGRESS: at sentence #137000, processed 32583097 words, keeping 1763421 word types
2022-10-29 01:33:18,710 : INFO : PROGRESS: at sentence #138000, processed 32830442 words, keeping 1768176 word types
2022-10-29 01:33:18,797 : INFO : PROGRESS: at sentence #139000, processed 33107686 words, keeping 1772313 word types
2022-10-29 01:33:18,880 : INFO : PROGRESS: at sentence #140000, processed 33367722 words, keeping 1777971 word types
2022-10-29 01:33:18,975 : INFO : PROGRESS: at sentence #141000, processed 33638880 words, keeping 1783455 word types
2022-10-29 01:33:19,056 : INFO : PROGRESS: at sentence #142000, processed 33894033 words, keeping 1788358 word types
2022-10-29 01:33:19,139 : INFO : PROGRESS: at sentence #143000, processed 34137822 words, keeping 1794083 word types
2022-10-29 01:33:19,229 : INFO : PROGRESS: at sentence #144000, 

2022-10-29 01:33:24,739 : INFO : PROGRESS: at sentence #207000, processed 51295166 words, keeping 2028477 word types
2022-10-29 01:33:24,803 : INFO : PROGRESS: at sentence #208000, processed 51505058 words, keeping 2030141 word types
2022-10-29 01:33:24,899 : INFO : PROGRESS: at sentence #209000, processed 51813223 words, keeping 2033000 word types
2022-10-29 01:33:24,995 : INFO : PROGRESS: at sentence #210000, processed 52151820 words, keeping 2034918 word types
2022-10-29 01:33:25,091 : INFO : PROGRESS: at sentence #211000, processed 52433583 words, keeping 2037671 word types
2022-10-29 01:33:25,192 : INFO : PROGRESS: at sentence #212000, processed 52733106 words, keeping 2040843 word types
2022-10-29 01:33:25,286 : INFO : PROGRESS: at sentence #213000, processed 53016472 words, keeping 2043433 word types
2022-10-29 01:33:25,389 : INFO : PROGRESS: at sentence #214000, processed 53321998 words, keeping 2046460 word types
2022-10-29 01:33:25,469 : INFO : PROGRESS: at sentence #215000, 

2022-10-29 01:33:30,864 : INFO : PROGRESS: at sentence #278000, processed 69747999 words, keeping 2179017 word types
2022-10-29 01:33:30,961 : INFO : PROGRESS: at sentence #279000, processed 69971989 words, keeping 2180883 word types
2022-10-29 01:33:31,052 : INFO : PROGRESS: at sentence #280000, processed 70247014 words, keeping 2182347 word types
2022-10-29 01:33:31,156 : INFO : PROGRESS: at sentence #281000, processed 70559737 words, keeping 2184817 word types
2022-10-29 01:33:31,224 : INFO : PROGRESS: at sentence #282000, processed 70785045 words, keeping 2185546 word types
2022-10-29 01:33:31,283 : INFO : PROGRESS: at sentence #283000, processed 70964129 words, keeping 2186063 word types
2022-10-29 01:33:31,338 : INFO : PROGRESS: at sentence #284000, processed 71125989 words, keeping 2186294 word types
2022-10-29 01:33:31,383 : INFO : PROGRESS: at sentence #285000, processed 71263883 words, keeping 2186297 word types
2022-10-29 01:33:31,452 : INFO : PROGRESS: at sentence #286000, 

...

2022-10-29 01:37:44,343 : INFO : EPOCH 0 - PROGRESS: at 3.01% examples, 32569 words/s, in_qsize 7, out_qsize 0
2022-10-29 01:37:45,796 : INFO : EPOCH 0 - PROGRESS: at 3.03% examples, 32507 words/s, in_qsize 7, out_qsize 0
2022-10-29 01:37:46,964 : INFO : EPOCH 0 - PROGRESS: at 3.05% examples, 32516 words/s, in_qsize 7, out_qsize 0
2022-10-29 01:37:48,056 : INFO : EPOCH 0 - PROGRESS: at 3.08% examples, 32539 words/s, in_qsize 7, out_qsize 0
2022-10-29 01:37:49,121 : INFO : EPOCH 0 - PROGRESS: at 3.09% examples, 32567 words/s, in_qsize 7, out_qsize 0
2022-10-29 01:37:50,262 : INFO : EPOCH 0 - PROGRESS: at 3.12% examples, 32582 words/s, in_qsize 7, out_qsize 0
2022-10-29 01:37:51,343 : INFO : EPOCH 0 - PROGRESS: at 3.15% examples, 32610 words/s, in_qsize 7, out_qsize 0
2022-10-29 01:37:52,442 : INFO : EPOCH 0 - PROGRESS: at 3.17% examples, 32632 words/s, in_qsize 7, out_qsize 0
2022-10-29 01:37:53,483 : INFO : EPOCH 0 - PROGRESS: at 3.20% examples, 32662 words/s, in_qsize 7, out_qsize 0
2

2022-10-29 01:39:14,724 : INFO : EPOCH 0 - PROGRESS: at 4.67% examples, 31791 words/s, in_qsize 7, out_qsize 0
2022-10-29 01:39:15,816 : INFO : EPOCH 0 - PROGRESS: at 4.70% examples, 31811 words/s, in_qsize 7, out_qsize 0
2022-10-29 01:39:16,861 : INFO : EPOCH 0 - PROGRESS: at 4.73% examples, 31837 words/s, in_qsize 7, out_qsize 0
2022-10-29 01:39:17,969 : INFO : EPOCH 0 - PROGRESS: at 4.75% examples, 31854 words/s, in_qsize 7, out_qsize 0
2022-10-29 01:39:19,186 : INFO : EPOCH 0 - PROGRESS: at 4.78% examples, 31856 words/s, in_qsize 7, out_qsize 0
2022-10-29 01:39:20,331 : INFO : EPOCH 0 - PROGRESS: at 4.80% examples, 31869 words/s, in_qsize 7, out_qsize 0
2022-10-29 01:39:21,793 : INFO : EPOCH 0 - PROGRESS: at 4.82% examples, 31839 words/s, in_qsize 7, out_qsize 0
2022-10-29 01:39:23,349 : INFO : EPOCH 0 - PROGRESS: at 4.84% examples, 31798 words/s, in_qsize 7, out_qsize 0
2022-10-29 01:39:24,492 : INFO : EPOCH 0 - PROGRESS: at 4.87% examples, 31810 words/s, in_qsize 7, out_qsize 0
2

2022-10-29 01:40:46,113 : INFO : EPOCH 0 - PROGRESS: at 6.19% examples, 30862 words/s, in_qsize 7, out_qsize 0
2022-10-29 01:40:47,454 : INFO : EPOCH 0 - PROGRESS: at 6.21% examples, 30854 words/s, in_qsize 7, out_qsize 0
2022-10-29 01:40:48,921 : INFO : EPOCH 0 - PROGRESS: at 6.24% examples, 30836 words/s, in_qsize 7, out_qsize 0
2022-10-29 01:40:49,933 : INFO : EPOCH 0 - PROGRESS: at 6.26% examples, 30831 words/s, in_qsize 7, out_qsize 0
2022-10-29 01:40:50,995 : INFO : EPOCH 0 - PROGRESS: at 6.27% examples, 30791 words/s, in_qsize 6, out_qsize 1
2022-10-29 01:40:52,229 : INFO : EPOCH 0 - PROGRESS: at 6.30% examples, 30795 words/s, in_qsize 7, out_qsize 0
2022-10-29 01:40:53,745 : INFO : EPOCH 0 - PROGRESS: at 6.33% examples, 30774 words/s, in_qsize 7, out_qsize 0
2022-10-29 01:40:55,125 : INFO : EPOCH 0 - PROGRESS: at 6.35% examples, 30764 words/s, in_qsize 8, out_qsize 0
2022-10-29 01:40:56,433 : INFO : EPOCH 0 - PROGRESS: at 6.38% examples, 30761 words/s, in_qsize 7, out_qsize 0
2

2022-10-29 01:42:18,054 : INFO : EPOCH 0 - PROGRESS: at 7.88% examples, 30726 words/s, in_qsize 7, out_qsize 0
2022-10-29 01:42:19,468 : INFO : EPOCH 0 - PROGRESS: at 7.91% examples, 30717 words/s, in_qsize 7, out_qsize 0
2022-10-29 01:42:20,760 : INFO : EPOCH 0 - PROGRESS: at 7.94% examples, 30693 words/s, in_qsize 7, out_qsize 0
2022-10-29 01:42:22,132 : INFO : EPOCH 0 - PROGRESS: at 7.97% examples, 30665 words/s, in_qsize 7, out_qsize 0
2022-10-29 01:42:23,765 : INFO : EPOCH 0 - PROGRESS: at 8.01% examples, 30641 words/s, in_qsize 7, out_qsize 0
2022-10-29 01:42:25,425 : INFO : EPOCH 0 - PROGRESS: at 8.03% examples, 30613 words/s, in_qsize 7, out_qsize 0
2022-10-29 01:42:26,987 : INFO : EPOCH 0 - PROGRESS: at 8.05% examples, 30594 words/s, in_qsize 6, out_qsize 1
2022-10-29 01:42:28,188 : INFO : EPOCH 0 - PROGRESS: at 8.06% examples, 30599 words/s, in_qsize 7, out_qsize 0
2022-10-29 01:42:29,252 : INFO : EPOCH 0 - PROGRESS: at 8.09% examples, 30614 words/s, in_qsize 8, out_qsize 0
2

2022-10-29 01:43:49,877 : INFO : EPOCH 0 - PROGRESS: at 9.64% examples, 30744 words/s, in_qsize 7, out_qsize 0
2022-10-29 01:43:51,283 : INFO : EPOCH 0 - PROGRESS: at 9.66% examples, 30736 words/s, in_qsize 7, out_qsize 0
2022-10-29 01:43:52,643 : INFO : EPOCH 0 - PROGRESS: at 9.68% examples, 30732 words/s, in_qsize 8, out_qsize 0
2022-10-29 01:43:54,084 : INFO : EPOCH 0 - PROGRESS: at 9.71% examples, 30722 words/s, in_qsize 7, out_qsize 0
2022-10-29 01:43:55,618 : INFO : EPOCH 0 - PROGRESS: at 9.73% examples, 30708 words/s, in_qsize 7, out_qsize 0
2022-10-29 01:43:56,797 : INFO : EPOCH 0 - PROGRESS: at 9.75% examples, 30714 words/s, in_qsize 7, out_qsize 0
2022-10-29 01:43:58,223 : INFO : EPOCH 0 - PROGRESS: at 9.77% examples, 30705 words/s, in_qsize 7, out_qsize 0
2022-10-29 01:43:59,463 : INFO : EPOCH 0 - PROGRESS: at 9.79% examples, 30707 words/s, in_qsize 7, out_qsize 0
2022-10-29 01:44:00,564 : INFO : EPOCH 0 - PROGRESS: at 9.82% examples, 30718 words/s, in_qsize 7, out_qsize 0
2

## Creating sequence database

Based on the trained model, let's create a sequence database using the [ScaNN engine](https://arxiv.org/abs/1908.10396).

In [ ]:
!ness build_database \
  --input data/pdb_seqres_proteins_only.fasta  \
  --model data/ness_pdb.model \
  --output data/ness_pdb.db \
  --debug

## Running a search against the database

With the formated database, we can compare a novel sequence to identify the most similar entries. To evaluate it, let's compare the protein sequence of the SARS-Cov-2 Spike protein and perform a multiple sequence alignment (MSA), comparing the results with those generated based on BLAST hits.

In [ ]:
!wget -O data/spike.fasta https://rest.uniprot.org/uniprotkb/P0DTC2.fasta

In [ ]:
!cat data/spike.fasta 

In [ ]:
%%time

!ness search \
  --input data/spike.fasta \
  --database data/ness_pdb.db \
  --output data/ness.csv \
  --hits 50

In [ ]:
!cat data/ness.csv

In [ ]:
df_ness_hits = pd.read_csv('data/ness.csv')
ness_hit_ids = list(df_ness_hits['subject'].head(50))

with open('data/ness_msa_inputs.fasta', 'w') as writer:
    for record in SeqIO.parse(open('data/spike.fasta'), 'fasta'):
        SeqIO.write([record], writer, 'fasta')
    for record in SeqIO.parse(open('data/pdb_seqres.fasta'), 'fasta'):
        if record.id in ness_hit_ids:
            SeqIO.write([record], writer, 'fasta')

In [ ]:
!muscle \
    -in data/ness_msa_inputs.fasta \
    -out data/ness_msa_output.html \
    -html

## Comparing top hits to BLAST

Now, let's compare the results obtained using NESS to those obtained using BLAST.

In [ ]:
!makeblastdb \
    -in data/pdb_seqres_proteins_only.fasta \
    -out data/pdb.blastdb \
    -dbtype prot

!blastp \
    -query data/spike.fasta \
    -db data/pdb.blastdb \
    -out data/blast.xml \
    -outfmt 5

Just like in the case of NESS, here we are going to select just to the top 10 results, as ranked by BLAST.

In [ ]:
blast_hits = NCBIXML.parse(open('data/blast.xml'))
blast_hits_ids = []
blast_hits_identity = []

for record in blast_hits:
    for alignment in record.alignments:
        blast_hits_ids.append(alignment.hit_def.split(' ')[0])
        blast_hits_identity.append(alignment.hsps[0].identities / alignment.hsps[0].align_length)
        if len(blast_hits_ids) == 50:
            break

with open('data/blast_msa_inputs.fasta', 'w') as writer:
    for record in SeqIO.parse(open('data/spike.fasta'), 'fasta'):
        SeqIO.write([record], writer, 'fasta')
        
    for record in SeqIO.parse(open('data/pdb_seqres.fasta'), 'fasta'):
        if record.id in blast_hits_ids:
            SeqIO.write([record], writer, 'fasta')

In [ ]:
len(blast_hits_ids)

In [ ]:
!muscle \
    -in data/blast_msa_inputs.fasta \
    -out data/blast_msa_output.html \
    -html 

The results generated by MUSCLE based on the hits detected by NESS and BLAST can be compared using the [AlignStat web server](http://alignstat.science.latrobe.edu.au/). From the PDB sequence dump obtained in October 20 (2022), the following report was generated. 

As we can see, the hits selected by NESS produced a better MSA when comparing to those selected by BLAST, probably because NESS considers an embedding representation of the sequence as a whole (global), instead of a sub-sequence of higher similarity (local), and uses a "semantically" representation that may also provide indirect information regarding the structure.


In [ ]:
def run_all_scores(sequences: list) -> None:
    
    aligned_sequences = list(pair[1] for pair in sequences)
    sequences_id = list(pair[0] for pair in sequences)

    msa = MSA(aligned_sequences, sequences_id)

    metrics = {
        'non_gaps': PercentageOfNonGaps(msa).compute(),
        'conserved': PercentageOfTotallyConservedColumns(msa).compute(),
        'entropy': Entropy(msa).compute(),
        'sum_of_pairs_blosum62': SumOfPairs(msa, Blosum62()).compute(),
        'sum_of_pairs_pam250': SumOfPairs(msa, PAM250()).compute(),
        'star_blosum62': Star(msa, Blosum62()).compute(),
        'star_pam250': Star(msa, PAM250()).compute(),
    }
    
    return metrics

In [ ]:
for fasta_file in ['data/ness_msa_output.fasta', 'data/blast_msa_output.fasta']:
    print(fasta_file)
    fasta_sequences = [(r.id, str(r.seq)) for r in SeqIO.parse(open(fasta_file), 'fasta')]
    print(run_all_scores(fasta_sequences))
    print('*'*100)